<a href="https://colab.research.google.com/github/azizamirsaidova/abstractive_text_summarization/blob/main/Pegasus_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import dependencies.**

In [ ]:
!pip install transformers
!pip install sentencepiece
!git clone https://github.com/huggingface/transformers

import numpy as np 
import pandas as pd 
import nltk
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import AutoTokenizer
import torch

from google.colab import drive
drive.mount('/content/gdrive')

**Access the Dataset.**

In [2]:
path = '/content/drive/MyDrive/train.csv'
trainning_df = pd.read_csv(path, engine='python', error_bad_lines=False)

In [4]:
MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 5000

trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()

trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])

del trainning_df

In [5]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

**Run Pre-trained PEGASUS on CNN/DailyMail dataset.**

In [ ]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
#tokenizer = PegasusTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(df['source_text'], truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
assert (tgt_text == df['target_text'])